In [1]:
import pickle
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
test_pred5=pickle.load(open('HIST_gcn/output/allind12_label11/rolling2005-01-01/pred.pkl.test0','rb'))
# test_pred5

In [4]:
test_pred5.groupby(level='datetime').apply(lambda x: x['label'].corr(x['score'], method='spearman')).mean()*100
# 'rankIC:',

13.237063348632653

In [5]:
ic_TRA = test_pred5.groupby(level='datetime').apply(lambda x: x['label'].corr(x['score'], method='spearman'))
ic_TRA = ic_TRA.reset_index()
ic_TRA.columns = ['date','rankIC']
ic_TRA['y'] = ic_TRA.date.apply(lambda x: str(x)[:4])
ic_TRA.groupby('y').mean(numeric_only=True)*100

,rankIC
y,
2019,12.486587
2020,13.269208
2021,13.357096
2022,13.593376
2023,13.484624


In [6]:
test_pred5=pickle.load(open('HIST_newdata/output/all_HIST_label11to1_20090430/rolling2009-01-01/pred.pkl.test0','rb'))
# test_pred5

In [7]:
test_pred5.groupby(level='datetime').apply(lambda x: x['label'].corr(x['score'], method='spearman')).mean()*100
# 'rankIC:',

12.32164657074968

In [8]:
ic_TRA = test_pred5.groupby(level='datetime').apply(lambda x: x['label'].corr(x['score'], method='spearman'))
ic_TRA = ic_TRA.reset_index()
ic_TRA.columns = ['date','rankIC']
ic_TRA['y'] = ic_TRA.date.apply(lambda x: str(x)[:4])
ic_TRA.groupby('y').mean(numeric_only=True)*100

,rankIC
y,
2019,13.599719
2020,11.791470
2021,11.524808
2022,12.365510


In [2]:
datanum = 'all_'  # csi300_, csi800_, all
modelname = 'HISTdelpre'

In [15]:
# dir_ = datanum + modelname
# with open('HIST_newdata/output51/'+dir_+'/rolling2009-01-01/pred.pkl.test0','rb')as f: # 
#     pred1=pickle.load(f)
# pred1.columns = ['score1','label1']
# pred1

pred1 = pickle.load(open('HIST_gcn/output/allind12_label11/rolling2005-01-01/pred.pkl.test0','rb'))
pred1.columns = ['score1','label1']
# pred1

In [16]:
# dir_ = datanum + modelname + '_label2to1'
# with open('HIST_newdata/output51/'+dir_+'/rolling2009-01-01/pred.pkl.test0','rb')as f: # 
#     pred2=pickle.load(f)
# pred2.columns = ['score2','label2']
# pred2
pred2 = pickle.load(open('HIST_newdata/output/all_HIST_label11to1_20090430/rolling2009-01-01/pred.pkl.test0','rb'))
pred2.columns = ['score2','label2']
# pred2

In [70]:
dir_ = datanum + modelname + '_label6to1'
with open('HIST_newdata/output51/'+dir_+'/rolling2009-01-01/pred.pkl.test0','rb')as f: # 
    pred6=pickle.load(f)
pred6.columns = ['score6','label6']
# pred6

In [71]:
dir_ = datanum + modelname + '_label11to1'
with open('HIST_newdata/output51/'+dir_+'/rolling2009-01-01/pred.pkl.test0','rb')as f: # 
    pred11=pickle.load(f)
pred11.columns = ['score11','label11']
# pred11

In [17]:
pred1 = pred1.reset_index()
pred1['datetime'] = pd.to_datetime(pred1['datetime'])

In [18]:
pred_all = pd.merge(pred1, pred2.reset_index())
# print(1)
# pred_all = pd.merge(pred_all, pred6.reset_index())
# print(2)
# pred_all = pd.merge(pred_all, pred11.reset_index())
# print(3)

In [19]:
pred_all = pred_all.set_index(['datetime','instrument'])

In [20]:
ic_TRA = pred_all.groupby(level='datetime').apply(lambda x: x['label1'].corr(x['score1'], method='spearman'))
ic_TRA = ic_TRA.reset_index()
ic_TRA.columns = ['date','rankIC']
ic_TRA['y'] = ic_TRA.date.apply(lambda x: str(x)[:4])
ic_TRA.groupby('y').mean(numeric_only=True)*100

,rankIC
y,
2019,12.520195
2020,13.273701
2021,13.375385
2022,13.604293


In [21]:
ic_TRA = pred_all.groupby(level='datetime').apply(lambda x: x['label2'].corr(x['score2'], method='spearman'))
ic_TRA = ic_TRA.reset_index()
ic_TRA.columns = ['date','rankIC']
ic_TRA['y'] = ic_TRA.date.apply(lambda x: str(x)[:4])
ic_TRA.groupby('y').mean(numeric_only=True)*100

,rankIC
y,
2019,13.537174
2020,11.788561
2021,11.513951
2022,12.411381


In [75]:
# 计算4*4的ic
label_list =[ 1,2,6,11]
for i in label_list:
    for j in label_list:
        # eval('rank_ic'+str(i)+str(j)) 
        numtow = pred_all.groupby(level='datetime').apply(lambda x: x['label'+str(i)].corr(x['score'+str(j)], method='spearman')).mean()*100
        with open('ic.txt','a',encoding='utf-8') as f:
            f.write(str(numtow)+'\t')
    with open('ic.txt','a',encoding='utf-8') as f:
        f.write('\n')

In [99]:
# 计算分年度的ic
def year_ic(i):
    # i=11 # 1,2,6,11 score
    for j in label_list:
        rank_ic = pred_all.groupby(level='datetime').apply(lambda x: x['label'+str(i)].corr(x['score'+str(j)], method='spearman'))
        rank_ic = rank_ic.reset_index()
        rank_ic.columns = ['date','ic'+str(i)+str(j)]
        rank_ic['y'] = rank_ic.date.apply(lambda x: str(x)[:4])
        if j ==1:
            rank_ic_all = rank_ic.groupby('y').mean(numeric_only=True)*100
            rank_ic_all = rank_ic_all.reset_index()
        else:
            rank_ic_temp = rank_ic.groupby('y').mean(numeric_only=True)*100
            rank_ic_all = pd.merge(rank_ic_all ,rank_ic_temp.reset_index())
    rank_ic_all.drop('y', axis=1).to_csv('rank_ic'+str(i)+'.txt',sep='\t', index=None, header=None)

In [83]:
for i in [1,2,6,11]:
    year_ic(i)

In [ ]:
# 向量的自相关是个矩阵，所以我暂时算了均值

In [84]:
import statsmodels.tsa.api as smt
# acf = smt.stattools.acf(pred_all['score1'])
# print(acf)

In [85]:
pred2 = pred_all.reset_index().set_index(['instrument','datetime'])
# acf = smt.stattools.acf(pred2.loc['SH600004']['score1'])
# print(acf)

In [86]:
pred_re = pred_all.reset_index()
pred_re['year'] = pred_re.datetime.apply(lambda x: str(x)[:4])
# pred_re[pred_re['year']==str(2019)]

In [96]:
# scorenum = '11'   # # 1,2,6,11 score

In [97]:
def acf_i(scorenum):
    ylist = [2019,2020,2021,2022]
    for y in ylist:
        pred_cal = pred_re[pred_re['year']==str(y)]
        acf1 = 0
        acf2 = 0
        acf5 = 0
        acf10 = 0
        stock300 = pred_cal['instrument'].drop_duplicates()
        pred_cal = pred_cal.set_index(['instrument','datetime'])
        lens = len(stock300)
        for a in stock300:
            cal_ser = pred_cal.loc[a]
            if len(cal_ser)>10:
                acf = smt.stattools.acf(cal_ser['score'+str(scorenum)])
                acf1 += acf[1]
                acf2 += acf[2]
                acf5 += acf[5]
                acf10 += acf[10]
            else:
                lens = lens-1
        with open('acf'+str(scorenum)+'.txt','a',encoding='utf-8') as f:
            f.write(str(acf1/lens)+'\t')
            f.write(str(acf5/lens)+'\t')
            f.write(str(acf10/lens)+'\t')
            f.write('\n')
        # print(acf1/lens,acf5/lens,acf10/lens)
    acf1 = 0
    acf2 = 0
    acf5 = 0
    acf10 = 0
    stock300 = pred2.reset_index()['instrument'].drop_duplicates()
    lens = len(stock300)
    for a in stock300:
        cal_ser = pred2.loc[a]
        if len(cal_ser)>10:
            acf = smt.stattools.acf(cal_ser['score'+str(scorenum)])
            acf1 += acf[1]
            acf2 += acf[2]
            acf5 += acf[5]
            acf10 += acf[10]
        else:
            lens = lens-1
    with open('acf'+str(scorenum)+'.txt','a',encoding='utf-8') as f:
        f.write(str(acf1/lens)+'\t')
        f.write(str(acf5/lens)+'\t')
        f.write(str(acf10/lens)+'\t')
        f.write('\n')
# print(acf1/lens,acf5/lens,acf10/lens)


In [98]:
for i in [1,2,6,11]:
    acf_i(i)

In [4]:
import pickle

In [5]:

with open('HIST_gcn/output/all_ind2_label11to1_GCN_re/rolling2009-01-01/pred.pkl.test0','rb')as f: # 
    adja1=pickle.load(f)
adja1

score     label
datetime   instrument                    
2019-01-02 SH600000   -0.044163 -0.202549
           SH600004   -0.066936 -1.195594
           SH600006    0.080465 -0.120721
           SH600007   -0.074251 -0.320745
           SH600008   -0.025778 -0.354082
...                         ...       ...
2022-12-30 SZ301388   -0.169745 -0.076298
           SZ301389   -0.073216  0.225442
           SZ301391   -0.050165  1.678214
           SZ301396    0.005414 -0.666659
           SZ301398   -0.237900  0.293800

[3967046 rows x 2 columns]

In [5]:
adja1.to_csv('all_GCN_industry2_label11.csv')

In [6]:
np.identity(3)+np.identity(3) 

array([[2., 0., 0.],
       [0., 2., 0.],
       [0., 0., 2.]])

In [2]:
dir_ = 'all_ind1_label11to1_HGCN'
with open('HIST_gcn/output/'+dir_+'/newdata_pred.pkl','rb')as f: # 
    pred1=pickle.load(f)
pred1

score     label
datetime   instrument                    
2023-01-03 SH600000   -3.749544 -1.118733
           SH600009   -3.749825  1.360933
           SH600010   -3.749825 -1.245600
           SH600011   -3.749544 -1.718467
           SH600015   -3.749544 -1.337867
...                         ...       ...
2023-05-31 SZ300896   -3.729171 -0.248796
           SZ300919   -3.729171 -0.572809
           SZ300957   -3.729171  0.595953
           SZ300979   -3.729171  1.695284
           SZ300999   -3.729171 -0.063645

[29399 rows x 2 columns]

In [2]:
import torch
def ic_metric(pred, label, weight=None):
    EPS = 1e-9
    # pred
    if pred.dim() == 1:
        pred = pred.view(-1,1)

    # label
    if label.dim() == 1:
        label = label.view(-1,1)

    # 权重, 默认等权
    if weight is None:
        weight = torch.ones_like(label)

    # 加权IC
    weight = weight/weight.sum()
    label -= label.t().mm(weight)
    label /= (label.mul(label).t().mm(weight)+EPS).sqrt()
    pred = pred - pred.t().mm(weight)
    pred = pred / (pred.mul(pred).t().mm(weight)+EPS).sqrt()
    ic = pred.mul(label).t().mm(weight).squeeze()

    return ic

def ic_loss(pred, label, weight=None):
    return - ic_metric(pred, label, weight)

/home/zhangxinyi/anaconda3/envs/mypytorch38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
pred = torch.randn(3,1)
label = torch.randn(3,1)
ic_metric(pred, label)

tensor(-0.9993)